In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import utilities.utils as utils

dataSource = "../data/rotten_tomatoes_movies.csv"
# Read the csv source file that is located in the data folder
df = pd.read_csv(dataSource)
print("Dataframe head analysis \n", df.head())
print("\n Dataframe null value analysis \n", df.isnull().sum())
print("\n Unique Directos analysis \n", df['directors'].unique())
print("\n Dataframe information \n", df.describe())

# Clean up the data for processing
columns = ['movie_title', 'genres', 'directors', 'tomatometer_rating']
directorDf = utils.clean_dataset(columns, df)

# Get the amount of times a Director appears in the dataset
directorCountDf = directorDf.directors.value_counts().reset_index().rename(columns={"index": "value", 0: "count"})
print(directorCountDf)
# Get the amount of directors that appear only once and drop them from directorDf, else get the average rating for each director
for index in directorCountDf.index:
    if directorCountDf['count'][index] <= 1:
        director = directorCountDf['directors'][index]
        
        directorDf.drop(directorDf[directorDf['directors'] == director].index, inplace=True, axis=0)
        
# Get the average rating for each director that is still in the dataframe
avgRating = pd.DataFrame(utils.get_avg_rating(directorDf))
print("Average Ratings for director(s) \n", avgRating)


Dataframe head analysis 
                     rotten_tomatoes_link  \
0                              m/0814255   
1                              m/0878835   
2                                   m/10   
3                 m/1000013-12_angry_men   
4  m/1000079-20000_leagues_under_the_sea   

                                         movie_title  \
0  Percy Jackson & the Olympians: The Lightning T...   
1                                        Please Give   
2                                                 10   
3                    12 Angry Men (Twelve Angry Men)   
4                       20,000 Leagues Under The Sea   

                                          movie_info  \
0  Always trouble-prone, the life of teenager Per...   
1  Kate (Catherine Keener) and her husband Alex (...   
2  A successful, middle-aged Hollywood songwriter...   
3  Following the closing arguments in a murder tr...   
4  In 1866, Professor Pierre M. Aronnax (Paul Luk...   

                                   

c:\Users\nicholas.ayala\Repos\GenAITraining\pythonExercises\project\src\utilities\utils.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=0, inplace=True)
C:\Users\nicholas.ayala\AppData\Local\Temp\ipykernel_42776\1174239350.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  directorDf.drop(directorDf[directorDf['directors'] == director].index, inplace=True, axis=0)


Average Ratings for director(s) 
                            directors  tomatometer_rating
0                    Aaron Katz (II)           75.000000
1                       Aaron Norris            8.000000
2     Aaron Seltzer, Jason Friedberg            1.500000
3                      Aaron Woodley           23.500000
4                   Abbas Kiarostami           89.250000
...                              ...                 ...
2733                        Zheng Xu           63.500000
2734                    Ziad Doueiri           80.666667
2735                    Zoltan Korda           85.000000
2736                     Zoya Akhtar           86.000000
2737              Álex de la Iglesia           67.500000

[2738 rows x 2 columns]


In [9]:
# Lets show a plot to show the average ratings for the director(s)
# Lets begin by creating some buckets divisible by 10
bucket10, bucket20, bucket30, bucket40, bucket50, bucket60, bucket70, bucket80, bucket90 = ([] for i in range(9))

# print(avgRating.head())
for index, row in avgRating.iterrows():
    print(row['directors'], row['tomatometer_rating'])
    rowValue = row['directors'], row['tomatometer_rating']
    print(rowValue)
    if row['tomatometer_rating'] <= 10:
        bucket10.append(rowValue)
        print(bucket10)
        input("holding...")
    
print(bucket10)
# Lets convert the avgRating series to numpy array
# avgRatingNp = avgRating.to_numpy()
# x = avgRatingNp[:, 0]
# y = avgRatingNp[:, -1]
# # print("This is x: \n", x)
# # print("\n This is y: \n", y)

# # Construct the scatterplot
# plt.title("Average Rating per Director")
# plt.xlabel("Directors")
# plt.ylabel("Avg Rating")
# plt.scatter(x, y, alpha=0.5)
# plt.show()

Aaron Katz (II) 75.0
('Aaron Katz (II)', 75.0)
Aaron Norris 8.0
('Aaron Norris', 8.0)
[('Aaron Norris', 8.0)]
Aaron Seltzer, Jason Friedberg 1.5
('Aaron Seltzer, Jason Friedberg', 1.5)
[('Aaron Norris', 8.0), ('Aaron Seltzer, Jason Friedberg', 1.5)]


KeyboardInterrupt: Interrupted by user